# Visualizing pipelines of filters 

In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
import cv2
from glob import glob

pipelines = {
    "Standard" : [
        lambda img : cv2.cvtColor(img, cv2.COLOR_BGR2RGB), # Warning: original pipeline skips this step!
        lambda img : cv2.cvtColor(img, cv2.COLOR_RGBA2RGB),
        lambda img : cv2.cvtColor(img, cv2.COLOR_BGR2GRAY),
        lambda img : cv2.threshold(img,127,255,cv2.THRESH_BINARY)[1],
        # lambda img : cv2.findContours(img, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_SIMPLE), # This filter is a no-op
    ],
    # TODO Fix second image overlapping
    # "Old pipeline" : [
    #     lambda img : cv2.cvtColor(img, cv2.COLOR_BGR2RGB),
    #     lambda img : cv2.pyrMeanShiftFiltering(img, sp=10, sr=100),
    #     lambda img : cv2.cvtColor(img, cv2.COLOR_BGR2GRAY),
    #     lambda img : cv2.threshold(img,127,255,cv2.THRESH_BINARY)[1],
    # ],
}

original_image_names = glob("dataset_cc2/*")
original_images = [cv2.imread(x) for x in original_image_names]
# original_images = original_images[:1] # TODO

fig = make_subplots(
    rows=max([len(pipeline) for pipeline in pipelines.values()]) + 1,
    cols=len(pipelines)*len(original_images),
    column_titles = [
        f"{pipeline_name}<br>{image_name.split('/')[1]}"
        for pipeline_name in pipelines.keys()
        for image_name in original_image_names
    ])

for image_idx, original_image in enumerate(original_images):
    for pipeline_idx, (_, pipeline) in enumerate(pipelines.items()):
        intermediates = [np.copy(original_image)]
        for filter in pipeline:
            intermediates.append(filter(intermediates[-1]))

        for intermediate_idx, img in enumerate(intermediates):
            fig.add_trace(
                px.imshow(img).data[0],
                row=intermediate_idx + 1,
                col=pipeline_idx + image_idx + 1)

fig.update_xaxes(visible=False, showticklabels=False)
fig.update_yaxes(visible=False, showticklabels=False)
fig.update_coloraxes(showscale=False)
fig.update_layout(height=1000, width=2500)
fig.show()